# BRDM: Brightness Restoration in Underwater Images from Diffusion Models

Este notebook foi projetado para testar e avaliar este modelo e as técnicas utilizadas em sua construção. O método utilizado aqui é destinado à pesquisa na área de aprimoramento de imagens. A principal questão é saber se é possível que um modelo de difusão aprenda a melhorar uma imagem subaquática, particularmente suas características de brilho. O objetivo principal é fornecer uma imagem como entrada e um valor de brilho, e o modelo gerará uma nova imagem com iluminação aprimorada. Em outras palavras, o método visa alcançar o aprimoramento controlável da iluminação em toda a imagem.

## Create the necessary envoiriment for the validation




In [ ]:
# make dir data if not exists
# download the dataset using gdown in this link
# descompact 
# move the dataset for the root
# make dir output if not exists
# mke dir ckpt
# make dir UIEB if not exists
# download UIEB ckpt  using gdown in this link
# descompact 
# move for the root
# make dir RUIE if not exists
# download RUIE ckpt using gdown in this link
# descompact
# move for the root
# make dir SUIM if not exists
# download SUIM ckpt using gdown in this link
# descompact
# move for the root

### Install requiriments or conda envoiriment 

### Download Dataset

In [ ]:
# Import necessary libraries
import os
import gdown
import zipfile

# Create directories if they don't exist
os.makedirs('data', exist_ok=True)
os.makedirs('output', exist_ok=True)
os.makedirs('ckpt', exist_ok=True)
os.makedirs('UIEB', exist_ok=True)
os.makedirs('RUIE', exist_ok=True)
os.makedirs('SUIM', exist_ok=True)

# Function to download, unzip and move files
def download_and_unzip(url, extract_to, move_to):
    # Download the file
    output_file = os.path.join(extract_to, 'temp.zip')
    gdown.download(url, output_file, quiet=False)
    
    # Unzip the file
    with zipfile.ZipFile(output_file, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    
    # Move the contents to the specified directory
    for item in os.listdir(extract_to):
        item_path = os.path.join(extract_to, item)
        os.rename(item_path, os.path.join(move_to, item))

    # Remove the temporary zip file
    os.remove(output_file)

# Fill in the links for download
dataset_link = 'INSERT_DATASET_LINK_HERE'
uie_b_link = 'INSERT_UIEB_CKPT_LINK_HERE'
ruie_link = 'INSERT_RUIE_CKPT_LINK_HERE'
suim_link = 'INSERT_SUIM_CKPT_LINK_HERE'

# Download, unzip and move the dataset
download_and_unzip(dataset_link, 'data', '/content')

# Download, unzip and move UIEB checkpoint
download_and_unzip(uie_b_link, 'UIEB', '/content')

# Download, unzip and move RUIE checkpoint
download_and_unzip(ruie_link, 'RUIE', '/content')

# Download, unzip and move SUIM checkpoint
download_and_unzip(suim_link, 'SUIM', '/content')

print("All files downloaded and organized successfully.")


### Download Checkpoints


Train from scratch, you can change setting in modelConfig 

```
!python main.py --dataset_path "" --dataset "UIEB" -- state "train" 
!python main.py --dataset_path "" --dataset "SUIM" -- state "train" 
```

Test The model with pretrained weights
```
!python main.py --pretrained_path  --dataset "SUIM" -- state "eval" --pretrained_path "1000.pt"
!python main.py --pretrained_path  --dataset "UIEB" -- state "eval" --pretrained_path "1000.pt"
```


Inference model with an selected image
```
!python main.py --pretrained_path  --dataset "SUIM" -- state "inference" --inference_image " "  --pretrained_path "1000.pt"
!python main.py --pretrained_path  --dataset "UIEB" -- state "inference" --inference_image " " --pretrained_path "1000.pt"
```

In [ ]:
!python main.py --dataset "UIEB" --state "inference" --pretrained_path "output/ckpt/UIEB/ckpt_1000_.pt" --wandb False --inference_image "/home/pdi_4/Documentos/Glown-Diffusion/data/UDWdata/UIEB/val/635_img_.png"

In [ ]:
!python main.py --dataset "RUIE" --state "inference" --pretrained_path "output/ckpt/RUIE/ckpt_1000_.pt" --wandb False --inference_image "/home/pdi_4/Documentos/Glown-Diffusion/data/UDWdata/RUIE/UIQS/val/A_655.jpg"

In [ ]:
!python main.py --dataset "UIEB" --state "inference" --pretrained_path "output/ckpt/UIEB/ckpt_1000_.pt" --wandb False --inference_image "/home/pdi_4/Documentos/Glown-Diffusion/data/UDWdata/UIEB/val/789_img_.png"

# Calculate Metrics